# Stacks Workflow for Extended Pink Data (Susitna and BC)

#### Stacks version 1.32

### Simplified steps:

Run process_radtags to clean up, demultiplex and rename the samples, replacing the barcode with the sample name. This will be done in either direction for the reads of both the original run and reruns, as the sequences are paired-end. 
Concatenate the directional reads of process_radtags and combine all the concatenated samples in one file. 
Run ustacks on each sample. 

Continue with Stacks:  sstacks and genotyping

### Data inventory:

Susitna populations
    
    PDISAP 2013 24 individuals
    PSPINK 2014 24 individuals
    
British Columbia populations 
    
    PLAKEL 2006 24 individuals
    PLAKEL 2007 24 individuals
      

### Background 

These samples were prepared for sequencing using bRAD, which sometimes leads to there being a barcode on the other end of the sequence- it isnt always next to the cut site. 

A separate factor for how we are running Stacks is that these are paired-end reads, which means that the sequence was read once from the front, or from the end of the fragment that has the cut site and then read again from the other side that does not have the cut site. The read from the side near the cut site is put into the file named R1 by Stacks, and the read from the side that is not near the cut site is put into R2. Anything that doesnt have a pair is put into the file that is called removed or .rem. The main reason that the reads are thrown out is if their other read was poor quality. Stacks also only keeps sequences that have the correct barcode and cut site, so it will artificially throw out the R1 with a barcode but no cut site. We don't use any of the sequences in the remainder files or the R2 sequences for snp calling.


## Running Process_radtags

In [ ]:
Here are the commands that I used to run process_radtags on these samples: 

process_radtags -1 ./DATA/PINK/RAD-SUSITNA_BC/RAW_DATA/Ogo1_S0_L006_R1_001.fastq -2 ./DATA/PINK/RAD-SUSITNA_BC/RAW_DATA/Ogo1_S0_L006_R2_001.fastq -o ./WORK/TARPEY/Exp_Pink_Pops/STACKS/PSBC1 -b ./WORK/TARPEY/Exp_Pink_Pops/STACKS/barcodesOgo1.txt -e sbfI -r -c -q -t 94 --filter_illumina -i fastq
process_radtags -1 ./DATA/PINK/RAD-SUSITNA_BC/RAW_DATA/Ogo1_S0_L006_R2_001.fastq -2 ./DATA/PINK/RAD-SUSITNA_BC/RAW_DATA/Ogo1_S0_L006_R1_001.fastq -o ./WORK/TARPEY/Exp_Pink_Pops/STACKS/PSBC2 -b ./WORK/TARPEY/Exp_Pink_Pops/STACKS/barcodesOgo1.txt -e sbfI -r -c -q -t 94 --filter_illumina -i fastq
process_radtags -1 ./DATA/PINK/RAD-SUSITNA_BC/RAW_DATA/Ogo1r_S60_L004_R1_001.fastq.gz -2 ./DATA/PINK/RAD-SUSITNA_BC/RAW_DATA/Ogo1r_S60_L004_R2_001.fastq.gz -o ./WORK/TARPEY/Exp_Pink_Pops/STACKS/PSBC1r -b ./WORK/TARPEY/Exp_Pink_Pops/STACKS/barcodesOgo1.txt -e sbfI -r -c -q -t 94 --filter_illumina -y fastq -i gzfastq
process_radtags -1 ./DATA/PINK/RAD-SUSITNA_BC/RAW_DATA/Ogo1r_S60_L004_R2_001.fastq.gz -2 ./DATA/PINK/RAD-SUSITNA_BC/RAW_DATA/Ogo1r_S60_L004_R1_001.fastq.gz -o ./WORK/TARPEY/Exp_Pink_Pops/STACKS/PSBC2r -b ./WORK/TARPEY/Exp_Pink_Pops/STACKS/barcodesOgo1.txt -e sbfI -r -c -q -t 94 --filter_illumina -y fastq -i gzfastq

To concatenate the output from process_radtags, I used a perl script that Garrett wrote called combine.pl. It takes the .1.fq files for one individual from each of the four process_radtags runs and concatenates them together in a new file, and the same with the .2.fq output files.

In [ ]:
#/usr/bin/perl -w

use strict;

my@files=`ls PSBC1`;
my%samples;
foreach my$file (@files){
	if($file=~/.fq/){
		my@info=split '\.', $file;
		#print "$info[0]\n";
		$samples{$info[0]}++;
	}
}

foreach my$sample (sort keys %samples){
	print "$sample\n";
	my$catCommand1="cat ./PSBC1/".$sample.".1.fq ./PSBC1r/".$sample.".1.fq ./PSBC2/".$sample.".1.fq ./PSBC2r/".$sample.".1.fq >./Combined/".$sample.".1.combined.fq";
	print "$catCommand1\n";
	`$catCommand1`;
	my$catCommand2="cat ./PSBC1/".$sample.".2.fq ./PSBC1r/".$sample.".2.fq ./PSBC2/".$sample.".2.fq ./PSBC2r/".$sample.".2.fq >./Combined/".$sample.".2.combined.fq";
	print "$catCommand2\n";
	`$catCommand2`;
}

To check that the samples concatenated correctly, I used a perl script that Garrett wrote that compares the number of lines in each of the original outputs from process_radtags for one sample before concatenaing and compares that to the number of lines in the concatenated file for each of the samples. It is called checkCombineLength.pl and it caught an error the first time I ran it through, which led me to re-run the concatenation script with a change to one of the file names.  (What is shown is the most current and accurate run.)

In [ ]:
#/usr/bin/perl -w
use strict;

my@files=`ls PSBC1`;
my%samples;
foreach my$file (@files){
	if($file=~/.fq/){
		my@info=split '\.', $file;
		$samples{$info[0]}++;
	}
}

print "Sample\tPSBC1\tPSBC1r\tPSBC2\tPSBC2r\tCombined\tMatch\n";
foreach my$sample (sort keys %samples){
	my$lengthCommand1="wc -l ./PSBC1/".$sample.".1.fq";
	my$lengthCommand2="wc -l ./PSBC1r/".$sample.".1.fq";
	my$lengthCommand3="wc -l ./PSBC2/".$sample.".1.fq";
	my$lengthCommand4="wc -l ./PSBC2r/".$sample.".1.fq";
	my$lengthCommand5="wc -l ./combined/".$sample.".1.combined.fq";
	my$lengthResult1=`$lengthCommand1`;
	my$lengthResult2=`$lengthCommand2`;
	my$lengthResult3=`$lengthCommand3`;
	my$lengthResult4=`$lengthCommand4`;
	my$lengthResult5=`$lengthCommand5`;
	chomp $lengthResult1;
	chomp $lengthResult2;
	chomp $lengthResult3;
	chomp $lengthResult4;
	chomp $lengthResult5;
	my$match1;
	if($lengthResult5==$lengthResult1+$lengthResult2+$lengthResult3+$lengthResult4){
		$match1=1;
	}else{
		$match1=0;
	}
	print "$sample.1.fq\t$lengthResult1\t$lengthResult2\t$lengthResult3\t$lengthResult4\t$lengthResult5\t$match1\n";
	
	my$lengthCommand6="wc -l ./PSBC1/".$sample.".2.fq";
	my$lengthCommand7="wc -l ./PSBC1r/".$sample.".2.fq";
	my$lengthCommand8="wc -l ./PSBC2/".$sample.".2.fq";
	my$lengthCommand9="wc -l ./PSBC2r/".$sample.".2.fq";
	my$lengthCommand10="wc -l ./combined/".$sample.".2.combined.fq";
	my$lengthResult6=`$lengthCommand6`;
	my$lengthResult7=`$lengthCommand7`;
	my$lengthResult8=`$lengthCommand8`;
	my$lengthResult9=`$lengthCommand9`;
	my$lengthResult10=`$lengthCommand10`;
	chomp $lengthResult6;
	chomp $lengthResult7;
	chomp $lengthResult8;
	chomp $lengthResult9;
	chomp $lengthResult10;
	my$match2;
	if($lengthResult10==$lengthResult6+$lengthResult7+$lengthResult8+$lengthResult9){
		$match2=1;
	}else{
		$match2=0;
	}
	print "$sample.2.fq\t$lengthResult6\t$lengthResult7\t$lengthResult8\t$lengthResult9\t$lengthResult10\t$match2\n";
}

## Running UStacks

These samples were shown to have been concatenated successfully. The next step on the Stacks pipeline is to run Ustacks.  To match previously run population samples, I used the same command on these pink samples. The parameters are these:

    -t the type of data file
    -f the path name to the concatenated samples
    -o the folder where the output should be placed
    -r use the removal algorithm and drop highly repetitive stacks
    --model_type bounded
    --bound_low 0
    --bound_high 0.05
    -i the SQL database number

It is imperative that the SQL number be unique to each individual. Previous runs ended with 777, so I started with 800. Here are the commands that were used to run ustacks on these samples, and the terminal output was copied to a txt file using | tee: 

In [ ]:
ustacks -t fastq -f Combined/PDISAP13_0004.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 800 
ustacks -t fastq -f Combined/PDISAP13_0008.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 801 
ustacks -t fastq -f Combined/PDISAP13_0009.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 802 
ustacks -t fastq -f Combined/PDISAP13_0012.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 803 
ustacks -t fastq -f Combined/PDISAP13_0013.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 804 
ustacks -t fastq -f Combined/PDISAP13_0033.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 805 
ustacks -t fastq -f Combined/PDISAP13_0056.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 806 
ustacks -t fastq -f Combined/PDISAP13_0064.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 807 
ustacks -t fastq -f Combined/PDISAP13_0066.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 808 
ustacks -t fastq -f Combined/PDISAP13_0067.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 809 
ustacks -t fastq -f Combined/PDISAP13_0069.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 810 
ustacks -t fastq -f Combined/PDISAP13_0070.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 811 
ustacks -t fastq -f Combined/PDISAP13_0071.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 812 
ustacks -t fastq -f Combined/PDISAP13_0073.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 813 
ustacks -t fastq -f Combined/PDISAP13_0077.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 814 
ustacks -t fastq -f Combined/PDISAP13_0078.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 815 
ustacks -t fastq -f Combined/PDISAP13_0079.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 816 
ustacks -t fastq -f Combined/PDISAP13_0080.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 817 
ustacks -t fastq -f Combined/PDISAP13_0081.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 818 
ustacks -t fastq -f Combined/PDISAP13_0090.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 819 
ustacks -t fastq -f Combined/PDISAP13_0091.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 820 
ustacks -t fastq -f Combined/PDISAP13_0092.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 821 
ustacks -t fastq -f Combined/PDISAP13_0094.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 822 
ustacks -t fastq -f Combined/PDISAP13_0095.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 823 
ustacks -t fastq -f Combined/PLAKEL06_0018.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 824 
ustacks -t fastq -f Combined/PLAKEL06_0020.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 825 
ustacks -t fastq -f Combined/PLAKEL06_0027.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 826 
ustacks -t fastq -f Combined/PLAKEL06_0028.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 827 
ustacks -t fastq -f Combined/PLAKEL06_0031.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 828 
ustacks -t fastq -f Combined/PLAKEL06_0035.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 829 
ustacks -t fastq -f Combined/PLAKEL06_0039.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 830 
ustacks -t fastq -f Combined/PLAKEL06_0043.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 831 
ustacks -t fastq -f Combined/PLAKEL06_0044.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 832 
ustacks -t fastq -f Combined/PLAKEL06_0047.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 833 
ustacks -t fastq -f Combined/PLAKEL06_0060.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 834 
ustacks -t fastq -f Combined/PLAKEL06_0061.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 835 
ustacks -t fastq -f Combined/PLAKEL06_0063.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 836 
ustacks -t fastq -f Combined/PLAKEL06_0066.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 837 
ustacks -t fastq -f Combined/PLAKEL06_0068.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 838 
ustacks -t fastq -f Combined/PLAKEL06_0069.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 839 
ustacks -t fastq -f Combined/PLAKEL06_0070.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 840 
ustacks -t fastq -f Combined/PLAKEL06_0073.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 841 
ustacks -t fastq -f Combined/PLAKEL06_0074.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 842 
ustacks -t fastq -f Combined/PLAKEL06_0075.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 843 
ustacks -t fastq -f Combined/PLAKEL06_0084.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 844 
ustacks -t fastq -f Combined/PLAKEL06_0092.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 845 
ustacks -t fastq -f Combined/PLAKEL06_0094.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 846 
ustacks -t fastq -f Combined/PLAKEL06_0101.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 847 
ustacks -t fastq -f Combined/PLAKEL07_0029.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 848 
ustacks -t fastq -f Combined/PLAKEL07_0031.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 849 
ustacks -t fastq -f Combined/PLAKEL07_0038.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 850 
ustacks -t fastq -f Combined/PLAKEL07_0039.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 851 
ustacks -t fastq -f Combined/PLAKEL07_0042.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 852 
ustacks -t fastq -f Combined/PLAKEL07_0046.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 853 
ustacks -t fastq -f Combined/PLAKEL07_0050.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 854 
ustacks -t fastq -f Combined/PLAKEL07_0056.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 855 
ustacks -t fastq -f Combined/PLAKEL07_0058.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 856 
ustacks -t fastq -f Combined/PLAKEL07_0160.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 857 
ustacks -t fastq -f Combined/PLAKEL07_0166.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 858 
ustacks -t fastq -f Combined/PLAKEL07_0167.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 859 
ustacks -t fastq -f Combined/PLAKEL07_0169.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 860 
ustacks -t fastq -f Combined/PLAKEL07_0172.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 861 
ustacks -t fastq -f Combined/PLAKEL07_0174.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 862 
ustacks -t fastq -f Combined/PLAKEL07_0175.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 863 
ustacks -t fastq -f Combined/PLAKEL07_0176.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 864 
ustacks -t fastq -f Combined/PLAKEL07_0178.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 865 
ustacks -t fastq -f Combined/PLAKEL07_0179.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 866 
ustacks -t fastq -f Combined/PLAKEL07_0180.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 867 
ustacks -t fastq -f Combined/PLAKEL07_0181.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 868 
ustacks -t fastq -f Combined/PLAKEL07_0190.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 869 
ustacks -t fastq -f Combined/PLAKEL07_0198.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 870 
ustacks -t fastq -f Combined/PLAKEL07_0200.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 871 
ustacks -t fastq -f Combined/PSPINK14_0001.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 872 
ustacks -t fastq -f Combined/PSPINK14_0004.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 873 
ustacks -t fastq -f Combined/PSPINK14_0006.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 874 
ustacks -t fastq -f Combined/PSPINK14_0007.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 875 
ustacks -t fastq -f Combined/PSPINK14_0009.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 876 
ustacks -t fastq -f Combined/PSPINK14_0012.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 877 
ustacks -t fastq -f Combined/PSPINK14_0014.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 878 
ustacks -t fastq -f Combined/PSPINK14_0015.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 879 
ustacks -t fastq -f Combined/PSPINK14_0018.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 880 
ustacks -t fastq -f Combined/PSPINK14_0020.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 881 
ustacks -t fastq -f Combined/PSPINK14_0021.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 882 
ustacks -t fastq -f Combined/PSPINK14_0024.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 883 
ustacks -t fastq -f Combined/PSPINK14_0028.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 884 
ustacks -t fastq -f Combined/PSPINK14_0029.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 885 
ustacks -t fastq -f Combined/PSPINK14_0035.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 886 
ustacks -t fastq -f Combined/PSPINK14_0037.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 887 
ustacks -t fastq -f Combined/PSPINK14_0039.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 888 
ustacks -t fastq -f Combined/PSPINK14_0041.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 889 
ustacks -t fastq -f Combined/PSPINK14_0042.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 890 
ustacks -t fastq -f Combined/PSPINK14_0043.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 891 
ustacks -t fastq -f Combined/PSPINK14_0044.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 892 
ustacks -t fastq -f Combined/PSPINK14_0045.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 893 
ustacks -t fastq -f Combined/PSPINK14_0051.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 894 
ustacks -t fastq -f Combined/PSPINK14_0052.1.combined.fq -r --model_type bounded --bound_low 0 --bound_high 0.05 -o Ustacks -i 895 

## Adding individuals to the Catalog

The catalog that we are adding the individuals to has 4 individuals from each of the 7 original populations and 7 female parents of the pink haploid crosses for the linkage maps that were built (see the notebook Building Catalog for more information).

We will add 4 individuals from each of these 4 new populations to the catalog that has already been built. Individuals were chosen based on their number of retained reads; the four individuals with the greatest number of retained reads in their populations were added to the catalog- the same way the individuals were chosen for the original catalog.

This is the code for adding those individuals to the catalog:

In [ ]:
cstacks -b 4 -p 6 -s Ustacks/PDISAP13_0008.1.combined -s Ustacks/PDISAP13_0033.1.combined -s Ustacks/PDISAP13_0090.1.combined -s Ustacks/PDISAP13_0069.1.combined -s Ustacks/PLAKEL06_0061.1.combined -s Ustacks/PLAKEL06_0043.1.combined -s Ustacks/PLAKEL06_0020.1.combined -s Ustacks/PLAKEL06_0060.1.combined -s Ustacks/PLAKEL07_0169.1.combined -s Ustacks/PLAKEL07_0175.1.combined -s Ustacks/PLAKEL07_0190.1.combined -s Ustacks/PLAKEL07_0050.1.combined -s Ustacks/PSPINK14_0021.1.combined -s Ustacks/PSPINK14_0009.1.combined -s Ustacks/PSPINK14_0020.1.combined -s Ustacks/PSPINK14_0018.1.combined --catalog RenamedEditedCatalog/batch_3.catalog -o Batch4Catalog -n 2 |& tee cstacksLOG.txt


## Running SStacks

With the catalog updated, the next step is running Sstacks, the portion of the pipeline that matches the putative loci that were identified for each sample in Ustacks against the putative loci named in the catalog. We will run the individuals in these 4 new populations as well as all the individuals in the original data set, as the catalog has changed since they were previously run. 

In [ ]:
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR10_0001 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR10_0002 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR10_0003 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR10_0004 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR10_0005_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR10_0006_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR10_0007 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR10_0008 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR10_0009 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR10_0010 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR10_0011 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR10_0012 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR10_0013_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR10_0014 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR10_0015 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR10_0016_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR10_0017 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR10_0018_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR10_0019 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR10_0020 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR10_0021 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR10_0022 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR10_0023 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR10_0024 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR10_0025 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR10_0026 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR10_0027_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR10_0028 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR10_0029_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR10_0030 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR10_0031_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR10_0032 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR11_0001_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR11_0002_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR11_0003_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR11_0004_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR11_0005_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR11_0006_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR11_0007_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR11_0008_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR11_0009_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR11_0010_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR11_0011_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR11_0013_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR11_0014_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR11_0015_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR11_0016_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR11_0017_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR11_0018_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR11_0019_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR11_0020_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR11_0021_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR11_0022_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR11_0023_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR11_0024_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR11_0025 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR11_0026_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR11_0027 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR11_0028 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR11_0029_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR11_0030_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR11_0031_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR11_0032_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PAMUR11_0033_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY09_0007 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY09_0008 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY09_0010_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY09_0011_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY09_0012_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY09_0013_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY09_0014_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY09_0015_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY09_0016_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY09_0017_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY09_0018_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY09_0019_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY09_0020 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY09_0021_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY09_0022_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY09_0023_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY09_0025_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY09_0026_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY09_0027_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY09_0028_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY09_0029_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY09_0030_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY09_0031 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY09_0032 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY09_0034_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY09_0035 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY09_0036_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY09_0037_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY09_0038_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY09_0041_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY09_0042_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY09_0043_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY10_0003 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY10_0004_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY10_0006_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY10_0007_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY10_0010_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY10_0011_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY10_0012_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY10_0014_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY10_0015_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY10_0017_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY10_0018_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY10_0020_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY10_0021_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY10_0022 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY10_0023_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY10_0024_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY10_0025 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY10_0026_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY10_0027_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY10_0029_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY10_0030_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY10_0032_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY10_0035 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY10_0036 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY10_0037 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY10_0038_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY10_0039 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY10_0040 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY10_0041 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY10_0042 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY10_0044 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PHAYLY10_0046_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE91T_0001 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE91T_0002 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE91T_0003 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE91T_0005 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE91T_0006 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE91T_0007 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE91T_0009 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE91T_0010 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE91T_0011 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE91T_0013 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE91T_0014 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE91T_0015 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE91T_0016 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE91T_0017 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE91T_0018 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE91T_0019 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE91T_0020 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE91T_0024 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE91T_0025 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE91T_0026 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE91T_0029 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE91T_0035 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE91T_0036 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE91T_0037 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE96T_0001 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE96T_0002 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE96T_0003 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE96T_0004 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE96T_0005 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE96T_0006 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE96T_0007 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE96T_0008 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE96T_0009 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE96T_0010 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE96T_0011 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE96T_0012 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE96T_0013 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE96T_0014 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE96T_0015 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE96T_0016 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE96T_0017 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE96T_0018 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE96T_0019 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE96T_0020 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE96T_0021 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE96T_0022 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE96T_0023 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKOPE96T_0024 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI06_0001_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI06_0002 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI06_0003_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI06_0004_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI06_0005_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI06_0006_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI06_0007_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI06_0008_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI06_0009_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI06_0010_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI06_0011_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI06_0012_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI06_0013 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI06_0014_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI06_0015_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI06_0016 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI06_0017 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI06_0018_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI06_0019_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI06_0020_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI06_0021_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI06_0022_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI06_0023 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI06_0024_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI06_0025 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI06_0026_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI06_0027_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI06_0028_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI06_0029 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI06_0030_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI06_0031_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI06_0032_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI07_0001_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI07_0002_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI07_0003_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI07_0004 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI07_0005_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI07_0006_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI07_0007_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI07_0008_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI07_0009 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI07_0010_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI07_0011_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI07_0012_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI07_0013 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI07_0014 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI07_0015 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI07_0016_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI07_0017 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI07_0018_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI07_0019 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI07_0020_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI07_0021_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI07_0022 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI07_0023 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI07_0024_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI07_0025_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI07_0026 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI07_0027_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI07_0028_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI07_0029_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI07_0030_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI07_0031_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PKUSHI07_0032_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME91_0001 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME91_0002 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME91_0003 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME91_0004 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME91_0005 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME91_0008 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME91_0009 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME91_0010 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME91_0011 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME91_0013 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME91_0014 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME91_0015 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME91_0016 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME91_0017 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME91_0018 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME91_0021 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME91_0025 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME91_0026 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME91_0027 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME91_0028 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME91_0030 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME91_0031 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME91_0032 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME91_0033 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME94_0001 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME94_0002 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME94_0003 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME94_0004 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME94_0005 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME94_0006 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME94_0007 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME94_0008 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME94_0009 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME94_0010 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME94_0011 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME94_0012 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME94_0013 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME94_0014 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME94_0015 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME94_0016 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME94_0017 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME94_0018 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME94_0019 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PNOME94_0020 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH03_0005 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH03_0016 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH03_0017 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH03_0019 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH03_0021 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH03_0024 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH03_0027 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH03_0029 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH03_0039 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH03_0043 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH03_0046 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH03_0063 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH03_0065 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH03_0067 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH03_0074 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH03_0075 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH03_0076 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH03_0078 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH03_0079 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH03_0081 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH03_0082 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH03_0088 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH03_0090 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH03_0095 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH96_0003 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH96_0004 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH96_0005 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH96_0006 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH96_0007 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH96_0008 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH96_0009 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH96_0010 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH96_0011 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH96_0012 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH96_0013 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH96_0014 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH96_0015 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH96_0016 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH96_0017 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH96_0018 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH96_0019 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH96_0021 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH96_0022 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH96_0023 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH96_0024 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH96_0025 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH96_0027 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PSNOH96_0028 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY09_0002 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY09_0003 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY09_0004 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY09_0006_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY09_0008_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY09_0009_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY09_0011 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY09_0012_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY09_0013_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY09_0014_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY09_0015 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY09_0016 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY09_0017 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY09_0018_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY09_0021 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY09_0022_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY09_0023 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY09_0024_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY09_0027 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY09_0031_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY09_0032_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY09_0033 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY09_0034_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY09_0035 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY09_0036 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY09_0037_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY09_0038 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY09_0040_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY09_0041 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY09_0042_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY09_0044_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY09_0046 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY12_0001_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY12_0002_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY12_0003_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY12_0004_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY12_0005_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY12_0006_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY12_0008_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY12_0009 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY12_0010_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY12_0011_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY12_0014 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY12_0015_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY12_0016_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY12_0019 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY12_0021_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY12_0022_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY12_0023_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY12_0024_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY12_0025_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY12_0026_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY12_0027 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY12_0030_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY12_0031_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY12_0032_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY12_0033 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY12_0034_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY12_0035_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY12_0036_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY12_0038 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY12_0039 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY12_0041_comb -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Pink_Populations/STACKS/Stacks_output/PTAUY12_0042 -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PDISAP13_0004.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PDISAP13_0008.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PDISAP13_0009.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PDISAP13_0012.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PDISAP13_0013.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PDISAP13_0033.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PDISAP13_0056.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PDISAP13_0064.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PDISAP13_0066.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PDISAP13_0067.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PDISAP13_0069.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PDISAP13_0070.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PDISAP13_0071.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PDISAP13_0073.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PDISAP13_0077.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PDISAP13_0078.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PDISAP13_0079.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PDISAP13_0080.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PDISAP13_0081.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PDISAP13_0090.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PDISAP13_0091.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PDISAP13_0092.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PDISAP13_0094.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PDISAP13_0095.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL06_0018.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL06_0020.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL06_0027.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL06_0028.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL06_0031.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL06_0035.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL06_0039.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL06_0043.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL06_0044.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL06_0047.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL06_0060.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL06_0061.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL06_0063.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL06_0066.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL06_0068.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL06_0069.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL06_0070.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL06_0073.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL06_0074.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL06_0075.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL06_0084.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL06_0092.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL06_0094.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL06_0101.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL07_0029.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL07_0031.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL07_0038.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL07_0039.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL07_0042.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL07_0046.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL07_0050.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL07_0056.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL07_0058.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL07_0160.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL07_0166.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL07_0167.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL07_0169.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL07_0172.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL07_0174.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL07_0175.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL07_0176.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL07_0178.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL07_0179.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL07_0180.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL07_0181.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL07_0190.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL07_0198.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PLAKEL07_0200.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PSPINK14_0001.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PSPINK14_0004.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PSPINK14_0006.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PSPINK14_0007.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PSPINK14_0009.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PSPINK14_0012.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PSPINK14_0014.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PSPINK14_0015.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PSPINK14_0018.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PSPINK14_0020.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PSPINK14_0021.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PSPINK14_0024.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PSPINK14_0028.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PSPINK14_0029.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PSPINK14_0035.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PSPINK14_0037.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PSPINK14_0039.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PSPINK14_0041.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PSPINK14_0042.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PSPINK14_0043.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PSPINK14_0044.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PSPINK14_0045.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PSPINK14_0051.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 
sstacks -b 4 -p 6 -c Exp_Pink_Pops/STACKS/Batch4Catalog/batch_4 -s Exp_Pink_Pops/STACKS/Ustacks/PSPINK14_0052.1.combined -o Exp_Pink_Pops/STACKS/Ustacks 

I did a quick check to make sure that all the Stacks output files were in the folder, so that there were no downstream errors. Here is a pivot table that has all the populations and the number of each type of output file (each sample has 4 Stacks output files at this junction)

    Populations  	Type of File				
        .alleles .matches .snps .tags Grand Total

    PAMUR10	    32	32	32	32	128
    PAMUR11	    32	32	32	32	128
    PDISAP13	   24	24	24	24	96
    PHAYLY09	   32	32	32	32	128
    PHAYLY10	   32	32	32	32	128
    PKOPE91T	   24	24	24	24	96
    PKOPE96T	   24	24	24	24	96
    PKUSHI06	   32	32	32	32	128
    PKUSHI07   	32	32	32	32	128
    PLAKEL06	   24	24	24	24	96
    PLAKEL07	   24	24	24	24	96
    PNOME91	    24	24	24	24	96
    PNOME94	    20	20	20	20	80
    PSNOH03	    24	24	24	24	96
    PSNOH96	    24	24	24	24	96
    PSPINK14	   24	24	24	24	96
    PTAUY09	    32	32	32	32	128
    PTAUY12	    32	32	32	32	128
    
    Grand Total	492	492	492	492	1968


## Creating whitelists for populations  

In [ ]:
#write_list_of_Loci.sh to be run as python write_list_of_loci.sh >listofLOCI.txt

loci_list=list(range(997122))

print '\n'.join(str(p) for p in loci_list)

Using the catalog tags file I could see that my new catalog has 997,121 loci in it. To run Garrett's script to break up the catalog into a set of equal size whitelists, I needed a list of all the loci in the catalog, so a file that has a list of 1-997,121. I could have edited the tags file, but I wrote a little python script to just make a file that had the loci names one on each line, see below. You need to open the output and delete the first line, then make sure the EOL is correct. After I had the list, I ran Garrett's perl script. 

In [ ]:
#/usr/bin/perl -w
#this splits up a list of all the loci into lists that are 35,000 loci long

use strict;

my$whitelistCount=1;
my$interval=35000;
my$outfile="whitelist_".$whitelistCount.".txt";
open(OUTFILE,">$outfile")||die "cannot open $outfile:$!";
my$count=0;
while(my$line=<>){
	chomp $line;
	$count++;
	print OUTFILE "$line\n";
	if($count%$interval==0){
		#print OUTFILE "$line\n";
		close OUTFILE;
		$whitelistCount++;
		$outfile="whitelist_".$whitelistCount.".txt";
		open(OUTFILE,">$outfile")||die "cannot open $outfile:$!";
		#$whitelistCount++;
		#print "$line\t$whitelistCount\n";
	}
	
}
close OUTFILE;

## Populations; first run for initial filtering

With a set of whitelists made, I prepped the rest of the populations command. I want to keep this run simple. I'm running every sample through at each locus to get the genotypes that will be used to do the first run of filtering in GenePop. From that round of filtering we will get another whitelist, or set of whitelists of the loci that passed the filtring step. At that point we will run populations in Stacks again with more outputs. 

Garrett wrote a script that automates the first run of populations with each of the whitelists we have created. It takes the population command that you want to run, as well as all the numbers of the whitelists that we have. It puts the results of each run through populations with each of the whitelists into its own folder named by the whitelist to prevent the output from being overwritten by the next run of populations. 

I wanted to run populations with a population map, so I made one and called it PopMap.txt. It just has each sample and the population that it belongs to, 18 populations total. 

    -P for the path to the directory containing the STACKS files. 
    -b batch ID for the run, here it is 4, the same as the catalog
    -t threads, here 6 because running on Ryan's
    -W the path to the whitelisted markers to include in the export. 

populations -P ./ -b 4 -t 6 -M PopMap.txt -W {garrett's script fills this in}

Garrett's script that runs each of the population commands in order of the whitelists:

In [ ]:
#/usr/bin/perl -w
use strict;

foreach my$i (1..29){
	my$command1="populations -P ./ -b 4 -t 6 -M PopMap.txt -W whitelist_".$i.".txt --genepop";
	my$command2="mkdir whitelist_".$i;
	my$command3="cp batch_4* whitelist_".$i;
	print "$command1\n";
	`$command1`;
	print "$command2\n";
	`$command2`;
	print "$command3\n";
	`$command3`;
}
